In [ ]:
# %%capture
# !pip install fasttext
# !pip install textblob
# !pip install spacy
# !pip install --upgrade gensim
#!pip install -U -q PyDrive
!pip install fasttext
!pip install nltk

In [ ]:
import torch 
import string
import json
import pandas as pd
import os
from google.colab import drive
import numpy as np
import random
from sklearn.model_selection import train_test_split
import urllib.request
from PIL import Image
import requests
from io import BytesIO
import torchvision.transforms as transforms
from gensim.models import FastText
import fasttext
import nltk
import re
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
nltk.download('wordnet')
from imblearn.over_sampling import RandomOverSampler
import csv

In [ ]:
drive.mount('/content/drive/')

In [ ]:
# Define category name
#####!!! please message on the chat before changing any of the paths below !!
category_name = 'meta_data'
path = f'/content/drive/My Drive/ai_project/dataset_processing/'
# Define paths for raw data, cleaned data, and data frequency files
# metadata_file_name = f'amazon_datasets/{category_name}.json'
# output_file_name = f'datasets_amazon_cleaned/{category_name}.csv'
# dataset_frequency_file_name = f'datasets_amazon_cleaned/title_word_breakdown/{category_name}_raw_title_frequency.csv'
cleaned_docs_path = f'{path}datasets_amazon_cleaned/all_metadata_files_upto25k/'
output_path = '/content/drive/My Drive/ai_project/dataset_processing/datasets_amazon_cleaned/dataset_3_components/images_loaded/data_cleaned_noIMG_pre_split.csv'
data_title_path = '/content/drive/My Drive/ai_project/dataset_processing/datasets_amazon_cleaned/title_testing/'

In [ ]:
# append the cleaned data to a DataFrame in Google Drive, searching everything in the datasets_amazon_cleaned folder

data_set = pd.DataFrame()

print(os.listdir(cleaned_docs_path))
for file in os.listdir(cleaned_docs_path):
  if file[-3:] == 'csv':
    sub_data = pd.read_csv(f'{cleaned_docs_path}{file}')
    data_set = data_set.append(sub_data)
    data_set = data_set.reset_index(drop=True)

**Further cleaning of the title**

In [ ]:
data_set.head(3)

In [ ]:
# Print DataFrame with categories, description, title, price
# getting rid of copy in main_cat
data_set = data_set.drop(['description','category','asin','title','title_token','title_lemmatize_tokens','title_stopwords_lemmatize_tokens'],axis = 1)
data_set

In [ ]:
data_set['main_cat'].value_counts()

In [ ]:
#dropna values
data_set = data_set.dropna(axis = 0 )
#combine luxurybeauty and all beauty,
data_set["main_cat"] = data_set["main_cat"].replace(['Luxury_Beauty'],'Beauty')
data_set["main_cat"] = data_set["main_cat"].replace(['All_Beauty'],'Beauty')
# #combine kindle with books 
data_set["main_cat"] = data_set["main_cat"].replace(['Kindle_Store'],'Books')
#combine prime pantry with grocery and gourmet food
data_set["main_cat"] = data_set["main_cat"].replace(['Prime_Pantry'],'Food')
data_set["main_cat"] = data_set["main_cat"].replace(['Grocery_and_Gourmet_Food'],'Food')
# #combine amazon fashion with clothing shoes and jewllery
data_set["main_cat"] = data_set["main_cat"].replace(['Amazon_Fashion'],'Clothing')
data_set["main_cat"] = data_set["main_cat"].replace(['Clothing_Shoes_and_Jewllery'],'Clothing')

data_set["main_cat"].value_counts()

In [ ]:
##These categories don't have enough datapoints with all the correct info 
# drop giftcards
data_set = data_set = data_set[data_set['main_cat'] != "Gift_Cards"]
# drop books 
data_set = data_set = data_set[data_set['main_cat'] != "Books"]
#drop software
data_set = data_set[data_set['main_cat'] != 'Software']
data_set["main_cat"].value_counts()

In [ ]:
#CDs and vinyl covers music, 2 music categories will be overkill
data_set = data_set[data_set['main_cat'] != 'Digital_Music']
#Books will need to be categorised but the title for books is highly variable so will rely on image and price instead for this category 
data_set



In [ ]:
# Downsample the number of Food and Clothing
def equalise_category(category_name, target_n, dataset):
  category_size = data_set[data_set['main_cat'] == category_name].count()[0]
  n = category_size - target_n
  index_to_remove = data_set[data_set['main_cat'] == category_name].sample(n).index
  data_set.drop(index_to_remove, inplace = True)
  return dataset


In [ ]:
data_set = equalise_category('Clothing',25000,data_set )
data_set = equalise_category('Food',25000,data_set )

In [ ]:
data_set['main_cat'].value_counts()

In [ ]:
data_set['number_of_images'] = data_set['imageURL'].apply(lambda x : len(x[1:-2].split(',')))

Categories which will need be oversampled after the data is split include (need 25,000 samples to ensure a balanced data_set)

*  Movies_and_TV                  21150
*  Digital_Music                  15513
*  Beauty                         11729
*  Video_Games                    10638
*  Applicances                     7360 












In [ ]:
#converting prices to floats and getting rid of the dollar 
#Maybe convert to gbp based on 2018 exchange rate if using it in the uk
data_set['price'] = data_set['price'].apply(lambda x:x.replace(',',''))
data_set['price'] = data_set['price'].apply(lambda x: float(x[1:]))

In [ ]:
# Add 'main_category' column with the first order list variable from the category column

#cat_x2 converts the main category string into an int
data_set["cat_x2"] = data_set["main_cat"]
data_set["cat_x2"] = data_set["cat_x2"].astype("category")
data_set['IntLabel'] = data_set["cat_x2"].cat.codes

# drop uneccesary category columns 
data_set = data_set = data_set.drop(['cat_x2'],axis=1)
data_set

**Cell to read images, ran locally as takes multiple days to load**

In [ ]:
# s = requests.Session()

# def convert_imageurl(image_url):
  
#   #gets the first image
#   try:
#     r = s.get(image_url[1:-1].split(', ')[0][1:-1])
#     # response = requests.get()
#     img = Image.open(BytesIO(r.content))
#     transform = transforms.Compose([
#     transforms.PILToTensor()
#       ])
#       # Convert the PIL image to Torch tensor
#     img_tensor = transform(img)
#     print('1')
#     return img_tensor
#   except:
#     print('first url does not work')
#     print(image_url[1:-1].split(', ')[0][1:-1])

#   try:
#     # if the image_url is actually a list of URLs
#     r = s.get(image_url[1:-2].split(', ')[0][1:-1])
#     img = Image.open(BytesIO(r.content))
#     transform = transforms.Compose([
#     transforms.PILToTensor()
#        ])
#     # Convert the PIL image to Torch tensor
#     img_tensor = transform(img)
#     print('1')
#     return img_tensor
#   except:
#     print('first URL is not a list')
#     print(image_url[1:-2].split(', ')[0][1:-1])
#   try :
#     # Trying 2nd image
#     r = s.get(image_url[1:-1].split(',')[1][1:-1])
#     img = Image.open(BytesIO(r.content))
#     transform = transforms.Compose([
#     transforms.PILToTensor()
#       ])
#      # Convert the PIL image to Torch tensor

#     print('1')
#     img_tensor = transform(img)
#     return img_tensor
#   except:
#     print('2nd image does not work')
#     print(image_url)
#     return torch.Tensor(0)
    

# data_set['imageURL'].apply(lambda x: convert_imageurl(x))


**Splitting the data into train and test sets**

In [ ]:
data_set.head(1)

In [ ]:

title_dataset = data_set[['cleaned_title','title_stopwords','title_lemmatize','title_stopwords_lemmatize','fastText_label','IntLabel']]

In [ ]:
#4 different preprocessing techniques which give 4 different titles
title_dataset.head(2)

In [ ]:
#Splitting the data in to a training set and a test set, 75% train, 25% test
X = title_dataset.drop("fastText_label",axis=1);
y = title_dataset['IntLabel']
#stratify the split data by the category for balanced datasets
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(X, y ,
                                                    stratify= y, 
                                                    test_size=0.25)

In [ ]:
X_test_title

In [64]:
def create_fast_text_file(data_set,label_set,column_header,test_or_train):
  '''
  The column_header can take, 'cleaned_title', 'title_stopwords','title_lemmatize','title_stopwords_lemmatize'
  This method creates a CSV file which can be used to train and evaluate a fastText model 

  '''
  test_data_dict = {'data':data_set[column_header].to_list(),'label':label_set.to_list()}
  test_data = pd.DataFrame(data = test_data_dict)
  test_data['input'] = '__label__' + test_data['label'].astype(str) + ' ' + test_data['data']
  test_data['input'].to_csv(f'{data_title_path}{column_header}_{test_or_train}_data.txt',index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")
  return test_data

In [49]:
# Creating 4 different files for the test data, each file has a different preprocessing step
create_fast_text_file(X_test_title,y_test_title,'cleaned_title','test')
create_fast_text_file(X_test_title,y_test_title,'title_stopwords','test')
create_fast_text_file(X_test_title,y_test_title,'title_lemmatize','test')
create_fast_text_file(X_test_title,y_test_title,'title_stopwords_lemmatize','test')
# # code for getting a datafile with each category 


In [50]:
## Resampling to increase to make the sample size the same for each category
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_title, y_train_title)
X_resampled['IntLabel'].value_counts()

11    18750
12    18750
19    18750
18    18750
2     18750
4     18750
8     18750
3     18750
6     18750
16    18750
10    18750
5     18750
17    18750
15    18750
7     18750
9     18750
13    18750
14    18750
0     18750
1     18750
Name: IntLabel, dtype: int64

In [51]:
# Creating the training files
# Creating 4 different files for the test data, each file has a different preprocessing step
create_fast_text_file(X_train_title,y_train_title,'cleaned_title','train')
create_fast_text_file(X_train_title,y_train_title,'title_stopwords','train')
create_fast_text_file(X_train_title,y_train_title,'title_lemmatize','train')
create_fast_text_file(X_train_title,y_train_title,'title_stopwords_lemmatize','train')

In [58]:
# A dictionary for the different cat_name to int labels
category_dict = dict(data_set[['IntLabel','main_cat']].value_counts().index.to_list())
category_dict

{10: 'Industrial_and_Scientific',
 18: 'Toys_and_Games',
 16: 'Sports_and_Outdoors',
 15: 'Pet_Supplies',
 14: 'Patio_Lawn_and_Garden',
 13: 'Office_Products',
 12: 'Musical Instruments',
 1: 'Arts_Crafts_and_Sewing',
 9: 'Home_and_Kitchen',
 8: 'Food',
 7: 'Electronics',
 6: 'Clothing',
 5: 'Cell_Phones_and_Accessories',
 4: 'CDs_and_Vinyl',
 2: 'Automotive',
 17: 'Tools_and_Home_Improvement',
 11: 'Movies_and_TV',
 3: 'Beauty',
 19: 'Video_Games',
 0: 'Applicances'}

# How well can the data be classified with a fast text classification model 

In [55]:
model_cleaned = fasttext.train_supervised(f'{data_title_path}cleaned_title_train_data.txt', wordNgrams = 2)
model_stopwords = fasttext.train_supervised(f'{data_title_path}title_stopwords_train_data.txt', wordNgrams = 2)
model_lemmatize = fasttext.train_supervised(f'{data_title_path}title_lemmatize_train_data.txt', wordNgrams = 2)
model_lemmatize_stopwords = fasttext.train_supervised(f'{data_title_path}title_stopwords_lemmatize_train_data.txt', wordNgrams = 2)

 

In [56]:
test_filenames = ['cleaned_title_test_data.txt','title_stopwords_test_data.txt','title_lemmatize_test_data.txt','title_stopwords_lemmatize_test_data.txt'] 
models = [model_cleaned,model_stopwords,model_lemmatize,model_lemmatize_stopwords]
results = {}
  
for n,file in enumerate(test_filenames):
  results[file] = models[n].test(f'{data_title_path}{file}')[1:3] 

In [57]:
# Stopwords has the highest overall accuracy, lets look at the accuarcy per category
results

{'cleaned_title_test_data.txt': (0.8500310586564913, 0.8500310586564913),
 'title_stopwords_test_data.txt': (0.8587541041796077, 0.8587541041796077),
 'title_lemmatize_test_data.txt': (0.8493033987044103, 0.8493033987044103),
 'title_stopwords_lemmatize_test_data.txt': (0.8571301801402077,
  0.8571301801402077)}

In [77]:
## checking 'title_stopwords_test_data.txt' accuracy per category 
category_test_set = create_fast_text_file(X_test_title,y_test_title,'title_stopwords','test')
for label in range(20):
  
  category_data = category_test_set[category_test_set['label'] == label]
  category_data['input'].to_csv(f'{data_title_path}title_test_individual_cats/label__{category_dict[label]}.txt',index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

In [78]:

results = {}

for n,file in enumerate(os.listdir(f'{data_title_path}title_test_individual_cats/')):
  results[file.split('__')[-1]] = model_stopwords.test(f'{data_title_path}title_test_individual_cats/{file}')[2] 

results
## not much title info is gained for Movies and TV, we will need to rely on the CNN for this category 

{'Applicances.txt': 0.8957992362247681,
 'Arts_Crafts_and_Sewing.txt': 0.86144,
 'Automotive.txt': 0.90784,
 'Beauty.txt': 0.803004438374872,
 'CDs_and_Vinyl.txt': 0.8288,
 'Cell_Phones_and_Accessories.txt': 0.95616,
 'Clothing.txt': 0.86864,
 'Electronics.txt': 0.89536,
 'Food.txt': 0.93904,
 'Home_and_Kitchen.txt': 0.83568,
 'Industrial_and_Scientific.txt': 0.8192,
 'Movies_and_TV.txt': 0.6874644414944054,
 'Musical Instruments.txt': 0.92976,
 'Office_Products.txt': 0.87168,
 'Patio_Lawn_and_Garden.txt': 0.86304,
 'Pet_Supplies.txt': 0.89664,
 'Sports_and_Outdoors.txt': 0.79408,
 'Tools_and_Home_Improvement.txt': 0.81648,
 'Toys_and_Games.txt': 0.81392,
 'Video_Games.txt': 0.863653483992467}

# Saving a file with the title_stopwords column 

In [84]:
output_data = data_set[['price','imageURL','main_cat','title_stopwords','IntLabel']]
output_path = f'/content/drive/MyDrive/ai_project/dataset_processing/datasets_amazon_cleaned/title_testing/output_before_imgs/dataset_title_preproccesed.csv'
output_data.to_csv(output_path)